In [ ]:
""" 
Goal, given monkey behavior, fit best MP and score.
(just single MP)

Also, try to optimize model, conditions on these programs

NBOTE: here is only extraction of MPs.
To score, refit, etc, see stroke_to_MP_loadAnd ...


NOTE:
REfitting libraries to monkey, moved to 
refitting_BPL_library_040621

"""

## PARSING, TESTING WAYS TO ASSIGN PARSES TO MOTOR PROGRAM

In [ ]:
# 1) use matlab function for converting parse to program (i.e, assigning strokes to primitives)


In [ ]:
import math
import imageio
import numpy as np
import matplotlib.pylab as plt

from pybpl.util import dist_along_traj
from pybpl.matlab.bottomup import generate_random_parses



def plot_stroke(ax, stk, color, lw=2):
    if len(stk) > 1 and dist_along_traj(stk) > 0.01:
        ax.plot(stk[:,0], -stk[:,1], color=color, linewidth=lw)
    else:
        ax.plot(stk[0,0], -stk[0,1], color=color, linewidth=lw, marker='.')

def plot_parse(ax, strokes, lw=2):
    ns = len(strokes)
    colors = ['r','g','b','m','c']
    for i in range(ns):
        plot_stroke(ax, strokes[i], colors[i], lw)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlim(0,105)
    ax.set_ylim(105,0)

def main():
    # load image to numpy binary array
    img = imageio.imread(f"{examples_dir}./image_H.jpg")
    img = np.array(img > 200)

    # generate random parses
    parses = generate_random_parses(img, seed=3)

    # plot parsing results
    nparse = len(parses)
    n = math.ceil(nparse/10)
    m = 10
    fig, axes = plt.subplots(n,m+1,figsize=(m+1, n))
    # first column
    axes[0,0].imshow(img, cmap=plt.cm.binary)
    axes[0,0].set_xticks([]); axes[0,0].set_yticks([])
    axes[0,0].set_title('Input')
    for i in range(1,n):
        axes[i,0].set_axis_off()
    # remaining_columns
    for i in range(n):
        for j in range(1,m+1):
            ix = i*m + (j-1)
            if ix >= nparse:
                axes[i,j].set_axis_off()
                continue
            plot_parse(axes[i,j], parses[ix])
    plt.subplots_adjust(hspace=0., wspace=0.)
    plt.show()

main()

In [ ]:
import os
# import cv2 # needed to create movie visualization of optimizaiton
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
# from tqdm import tqdm
import imageio
import torch
import torch.distributions as dist

from pybpl.library import Library
from pybpl.model import CharacterModel
from pybpl.objects.concept import CharacterType
from pybpl.objects.part import StrokeType
from pybpl.objects.relation import RelationIndependent, RelationAttachAlong
from pybpl import rendering

# library and type distribution
lib = Library('/data1/code/python/pyBPL/lib_data/', )

### PUTTING IT ALL TOGETHER

In [2]:
import pandas as pd
import pickle 
from pythonlib.dataset.dataset import Dataset
from pythonlib.bpl.strokesToProgram import *


In [3]:
def extractMPlist(path_list, params_preprocess = ["recenter", "spad_edges"]):
    for path in path_list:

        D = Dataset([path], None)

        D.preprocessGood(ver="", params=params_preprocess)

        if True:
            # maybe this better, keep it stable caross all epxts
            sketchpad_edges = np.array([[-260., 260.],[-260., 260.]])
        else:
            sketchpad_edges = D.Metadats[0]["sketchpad_edges"].T
        print(sketchpad_edges)

        ## returns params in lists.
        strokeslist = D.Dat["strokes_beh"].values
        trialcodelist = D.Dat["trialcode"].values
        save_checkpoints = [100, path]
        MPlist, scores = infer_MPs_from_strokes(strokeslist, trialcodelist, params_preprocess,
                                                sketchpad_edges, save_checkpoints=save_checkpoints)

        return MPlist, scores

In [ ]:
# [V1] automatic way to extract data
from pythonlib.tools.expttools import findPath

for animal in ["Pancho", "Red"]:
    path_list = findPath("/data2/analyses/database/", [[animal, "bias", "formodeling"]], return_without_fname=True)
    
    extractMPlist(path_list)

In [4]:
# [V1] automatic way to extract data
from pythonlib.tools.expttools import findPath

for animal in ["Pancho", "Red"]:
    path_list = findPath("/data2/analyses/database/", [[animal, "bias", "formodeling"]], return_without_fname=True)
    
    for path in path_list:

        D = Dataset([path])
        D.bpl_extract_and_save_motorprograms()


Searching using this string:
/data2/analyses/database//*Pancho*bias*formodeling*/**
-- Splitting off dir from fname
Found this many paths:
7
---
/data2/analyses/database/Pancho-biasdir2-formodeling-210406_014243
---
/data2/analyses/database/Pancho-biasdir3-formodeling-210406_014356
---
/data2/analyses/database/Pancho-biasdir4-formodeling-210406_014410
---
/data2/analyses/database/Pancho-biasdir5-formodeling-210406_014438
---
/data2/analyses/database/Pancho-biasdir6-formodeling-210406_014451
---
/data2/analyses/database/Pancho-biasdir7_1-formodeling-210406_014500
---
/data2/analyses/database/Pancho-biasdir7_2-formodeling-210406_014510
----------------
Currently loading: /data2/analyses/database/Pancho-biasdir2-formodeling-210406_014243
Loaded dataset, size:
1015
Loaded metadat:
{'sketchpad_edges': array([[-327.2, -327.2],
       [ 327.2,  429.6]])}
----
Resetting index
=== CLEANING UP self.Dat ===== 
ORIGINAL: online abort values
too_far_from_ink    196
hotkey_abort         79
Name: onl

Replaced self.Metadats with updated sketchpad...
[-x, -y; +x +y]
[[-254.09937537 -209.7180622 ]
 [ 254.09937537  209.7180622 ]]
Saving checkpoint
Saving checkpoint
Saving checkpoint
Saving checkpoint
Saving checkpoint
Saving checkpoint
----------------
Currently loading: /data2/analyses/database/Red-biasdir4-formodeling-210406_014401
Loaded dataset, size:
404
Loaded metadat:
{'sketchpad_edges': array([[-327.2, -327.2],
       [ 327.2,  429.6]])}
----
Resetting index
=== CLEANING UP self.Dat ===== 
ORIGINAL: online abort values
too_far_from_ink    79
Name: online_abort, dtype: int64
kept 325 out of 404
removed all cases with online abort not None
Deleted unused columns from self.Dat
Replaced self.Metadats with updated sketchpad...
[-x, -y; +x +y]
[[-242.64910259 -243.42648889]
 [ 242.64910259  243.42648889]]
Saving checkpoint
Saving checkpoint
Saving checkpoint
Saving checkpoint
Saving checkpoint
Saving checkpoint
----------------
Currently loading: /data2/analyses/database/Red-biasdir5

In [ ]:
# [V2] Give me path by hand (monkey strokes data

path_list = [
    "/data2/analyses/database/Red-lines5-formodeling-210329_005719",
    "/data2/analyses/database/Red-arc2-formodeling-210329_005550",
    "/data2/analyses/database/Red-shapes3-formodeling-210329_005200",
    "/data2/analyses/database/Red-figures89-formodeling-210329_005443",
    "/data2/analyses/database/Pancho-lines5-formodeling-210329_014835",
    "/data2/analyses/database/Pancho-arc2-formodeling-210329_014648",
    "/data2/analyses/database/Pancho-shapes3-formodeling-210329_002448",
    "/data2/analyses/database/Pancho-figures89-formodeling-210329_000418"
]

MPlist, scores = extractMPlist(path_list, params_preprocess)


In [ ]:
# === PLOT
for MPs, s in zip(MPlist, scores):
    plotMP(MPs[0], s[0])
#     plotMP(MPlist[1][0])

In [ ]:
MP = MPlist[0][0]
for param in MP.parameters():
    print(param.requires_grad)
    param.requires_grad=True
    print(param.shape)

In [ ]:
scoreMPs([MP[0] for MP in MPlist])

In [ ]:
# === SCORE
scoreMPs([x[0] for x in MPlist])

In [ ]:
# === SCORE
scoreMPs([x[0] for x in MPlist], use_hist=False)

In [ ]:
plt.figure()
plt.hist(score_all, 20)

In [ ]:
########### PLOT
# get data
import random
ind = random.choice(range(len(out_all)))
out = out_all[ind]
score = score_all[ind]

# Plot program
ctype = params2ctype(out[0])
plotMP(ctype)

# plot origianl strokes data
from pythonlib.drawmodel.strokePlots import plotDatStrokes
strokes = D.Dat["strokes_beh"].values[ind]
fig, ax = plt.subplots()
plotDatStrokes(strokes, ax, each_stroke_separate=True)
ax.set_title(f"score {score}");

### convert back to motor program

In [ ]:
if False:
    # get optimizable parameters
    params = ctype.parameters() + ctoken.parameters()
    lbs = ctype.lbs() + ctoken.lbs()
    ubs = ctype.ubs() + ctoken.ubs()